In [3]:
!nvidia-smi

Thu Nov 23 05:14:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
ROOT_DIR = '/content/gdrive/My Drive/Yolov8Developer/Project_Yolov8'

In [6]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.216 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.9/78.2 GB disk)


In [ ]:
import os
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml')  # build a new model from scratch
model = YOLO('/content/gdrive/MyDrive/Yolov8Developer/Project_Yolov8/yolov8n_customized.pt')  # load a pretrained model (recommended for training)

# Use the model
results = model.train(data=os.path.join(ROOT_DIR, "train.yaml"), epochs=1)  # train the model
results = model.val()  # evaluate model performance on the validation set
results = model.export(format='onnx')  # export the model to ONNX format

In [ ]:
#For GPU
import shutil

source_directory = '/content/runs'
target_directory = '/content/gdrive/My Drive/Yolov8Developer/Project_Yolov8/runs'

# Copy the directory, or copy the target directory if it already exists.
shutil.copytree(source_directory, target_directory, dirs_exist_ok=True)

In [ ]:
# For TPU or CPU
!scp -r /content/runs '/content/gdrive/My Drive/Yolov8Developer/Project_Yolov8'

# Colab to Drive (exchange)

In [ ]:
!mv datasets "/content/gdrive/My Drive/Yolov8Developer/Project_Yolov8"

In [ ]:
!cp -r "/content/gdrive/MyDrive/Yolov8Developer/Project_Yolov8/datasets" "/content/"

# Route

In [71]:
import os

for root, dirs, files in os.walk('/'):
    if 'train.py' in files:
        print(os.path.join(root, 'train.py'))

/usr/local/lib/python3.10/dist-packages/ultralytics/models/rtdetr/train.py
/usr/local/lib/python3.10/dist-packages/ultralytics/models/yolo/classify/train.py
/usr/local/lib/python3.10/dist-packages/ultralytics/models/yolo/pose/train.py
/usr/local/lib/python3.10/dist-packages/ultralytics/models/yolo/segment/train.py
/usr/local/lib/python3.10/dist-packages/ultralytics/models/yolo/detect/train.py
/usr/local/lib/python3.10/dist-packages/spacy/cli/train.py
/usr/local/lib/python3.10/dist-packages/dopamine/labs/atari_100k/train.py
/usr/local/lib/python3.10/dist-packages/dopamine/labs/tandem_dqn/train.py
/usr/local/lib/python3.10/dist-packages/dopamine/labs/environments/brax/train.py
/usr/local/lib/python3.10/dist-packages/dopamine/discrete_domains/train.py
/usr/local/lib/python3.10/dist-packages/dopamine/continuous_domains/train.py
/usr/local/lib/python3.10/dist-packages/transformers/commands/train.py
/tools/google-cloud-sdk/lib/surface/ai_platform/local/train.py


# Structure

In [ ]:
import torch

# Load model weights
weights = torch.load('/content/gdrive/MyDrive/Yolov8Developer/Project_Yolov8/yolov8n_customized.pt')

# Show Model Structure
print(weights.keys())

# Pre-trained

In [ ]:
# Download COCO val
import torch
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d datasets && rm tmp.zip  # unzip

In [ ]:
# Train YOLOv8n on COCO8 for 10 epochs
!yolo train model=yolov8n.pt data=coco8.yaml epochs=10 imgsz=640

In [ ]:
!yolo export model=/content/gdrive/MyDrive/Yolov8Developer/Yolov8/yolov8n.pt format=torchscript

In [55]:
import torch

# Load model weight files
weights_path = '/content/gdrive/MyDrive/Yolov8Developer/Project_Yolov8/yolov8n_customized.pt'
checkpoint = torch.load(weights_path)

# Modify the optimizer
checkpoint['optimizer'] = {
    'state': {},  # Status is set to null
    'param_groups': [
        {
            'lr': 0.0001,  # initial learning rate
            'momentum': 0.9,  # SGD momentum
            'betas': (0.9, 0.999),  # Setting the beta value
            'eps': 1e-08,  # Setting the epsilon value
            'weight_decay': 0.0005,  # weight decay
            'amsgrad': False,  # Whether to use AMSGrad
            'params': [],  # Parameter list, depending on the model
            'burn_in': 1000  # number of burn-in iterations
        }
    ]
}

# Save the modified weights back to the file
torch.save(checkpoint, weights_path)

In [63]:
import torch

# Load model weight files
weights_path = '/content/gdrive/MyDrive/Yolov8Developer/Project_Yolov8/yolov8n_customized.pt'
checkpoint = torch.load(weights_path)

# Modify the optimizer
checkpoint['train_args'] = {
    'state': {},  # Status is set to null
    'param_groups': [
        {
            'mode': 'train',
            'model': 'yolov8n.yaml',
            'data': 'coco.yaml',
            'epochs': 500,
            'patience': 50,
            'batch': 16,
            'imgsz': 640,
            'save': True,
            'cache': 'disk',
            'device': 0,
            'workers': 8,
            'project': 'YOLOv8',
            'name': 'yolov8n',
            'exist_ok': False,
            'pretrained': False,
            'optimizer': 'SGD',
            'verbose': False,
            'seed': 0,
            'deterministic': True,
            'single_cls': False,
            'image_weights': False,
            'rect': False,
            'cos_lr': False,
            'close_mosaic': 10,
            'resume': False,
            'overlap_mask': True,
            'mask_ratio': 4,
            'dropout': False,
            'val': True,
            'save_json': False,
            'save_hybrid': False,
            'conf': 0.001,
            'iou': 0.7,
            'max_det': 300,
            'half': True,
            'dnn': False,
            'plots': True,
            'source': 'ultralytics/assets/',
            'show': False,
            'save_txt': False,
            'save_conf': False,
            'save_crop': False,
            'hide_labels': False,
            'hide_conf': False,
            'vid_stride': 1,
            'line_thickness': 3,
            'visualize': False,
            'augment': False,
            'agnostic_nms': False,
            'retina_masks': False,
            'format': 'torchscript',
            'keras': False,
            'optimize': False,
            'int8': False,
            'dynamic': False,
            'simplify': False,
            'opset': 17,
            'workspace': 4,
            'nms': False,
            'lr0': 0.0001,
            'lrf': 0.0001,
            'momentum': 0.9,
            'weight_decay': 0.001,
            'warmup_epochs': 3.0,
            'warmup_momentum': 0.8,
            'warmup_bias_lr': 0.1,
            'box': 7.5,
            'cls': 0.5,
            'dfl': 1.5,
            'fl_gamma': 0.0,
            'label_smoothing': 0.0,
            'nbs': 64,
            'hsv_h': 0.015,
            'hsv_s': 0.7,
            'hsv_v': 0.4,
            'degrees': 0.0,
            'translate': 0.1,
            'scale': 0.5,
            'shear': 0.0,
            'perspective': 0.0,
            'flipud': 0.0,
            'fliplr': 0.5,
            'mosaic': 1.0,
            'mixup': 0.0,
            'copy_paste': 0.0,
            'cfg': None,
            'v5loader': True,
        }
    ]
}

# Save the modified weights back to the file
torch.save(checkpoint, weights_path)

In [ ]:
torch.load("/content/gdrive/MyDrive/Yolov8Developer/Project_Yolov8/yolov8n_customized.pt")
checkpoint['optimizer']

In [ ]:
torch.load("/content/gdrive/MyDrive/Yolov8Developer/Project_Yolov8/yolov8n_customized.pt")

In [ ]:
torch.load("/content/gdrive/MyDrive/Yolov8Developer/Project_Yolov8/yolov8n.pt")